### Enviar Mensagens pelo WhatsApp com Imagens V2

Este programa pega a lista de clientes do arquivo de vendas en Excel

A partir da folha "CLIENTES"
Percorre a lista e envia a mensagem da coluna "MENSAGEM"
Usando o telefone da coluna "TELEFONE"

Envia tambem as imagens selecionadas

Gera um relatorio de sucesso/fracasso para cada "CLIENTE"

In [1]:
# importar webdriver da biblioteca selenium para manipular navegadores
from selenium import webdriver

# importar blibiotecas necessarias do selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys

# importar biblioteca time para criar tempos de espera
import time

# importar datetime para guardar a data de hoje
from datetime import date

# importar biblioteca random para criar tempos de espera aleatorios
import random

# importar biblioteca pandas para manipular arquivos Excel
import pandas as pd

# pip install webdriver_manager
# bilioteca webdriver_manager atualiza automaticamente os drivers dos navegadores
from webdriver_manager.chrome import ChromeDriverManager

import urllib # libraria necessaria para transformar texto ascii em url (acentos, espaços, etc)

from tkinter import filedialog as fd

In [2]:
# definir tipos de arquivo de imagem para abrir
tipos = [('Arquivos de imagem PNG', '*.png'),('Arquivos de imagem JPG', '*.jpg'),('Outros tipos','*.*')]

# selecionar os arquivos de imagens, salvar na lista imagens
imagens = sorted(list(fd.askopenfilenames(title='Seleccione arquivos de imagem',filetypes=tipos)))

# abrir o arquivo de clientes
arquivo = fd.askopenfilename(title='Selecione o arquivo Excel')

# ler arquivo Excel, precisa tres colunas
contatos_df = pd.read_excel(arquivo, sheet_name='CLIENTES')

In [3]:
lista_enviados = []

In [4]:
# criar instancia do navegador Google Chrome
navegador = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\pgetar\.wdm\drivers\chromedriver\win32\106.0.5249.61]
C:\Users\pgetar\AppData\Local\Temp/ipykernel_17212/1362990109.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
# ir no site Whatsapp Web
navegador.get("https://web.whatsapp.com/")
time.sleep(5)

In [6]:
# autorizar com o celular o QR Code
# aguardar que carrege a pagina lateral com contatos
while len(navegador.find_elements(By.ID,"side")) < 1:
    time.sleep(5)

In [7]:
# já estamos com o login feito no whatsapp web
# para cada linha, obter o numero de registro e a mensagem do dataframe
for i, mensagem in enumerate(contatos_df['MENSAGEM']):

    if not pd.isnull(mensagem):
        # obter cliente e numero de telefone do dataframe
        nome = contatos_df.loc[i,"CLIENTE"]
        numero = contatos_df.loc[i, "TELEFONE"]
    
        # transformar mensagem de ascii em url text
        texto = urllib.parse.quote(f"{mensagem}")
    
        # construir o link
        link = f"https://web.whatsapp.com/send?phone={numero}&text={texto}"
    
        # tentar enviar o link, se o numero der errado, pular para o proximo
        try:
            # colocar o link
            navegador.get(link)
            time.sleep(7)
           
            # aguardar aparecer lista de contatos, indicando que podemos apertar o enter
            while len(navegador.find_elements(By.ID,"side")) < 1:
                time.sleep(7)
                
            # enviar o ENTER
            
            navegador.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p/span').send_keys(Keys.ENTER)
            
            # aguardar aparecer lista de contatos, indicando que podemos começar a enviar imagens
            time.sleep(3)

            # para cada imagen da lista
            for imagen in imagens:
                
                # find attachment button and click on it
                navegador.find_element(By.CSS_SELECTOR,"span[data-icon='clip']").click()
                time.sleep(1)

                # select to find element and send all keys
                navegador.find_element(By.CSS_SELECTOR,"input[type='file']").send_keys(imagen)
                time.sleep(1)

                # click on send attachment
                navegador.find_element(By.CSS_SELECTOR,"span[data-icon='send']").click()
                time.sleep(1)

            # imprimir no terminal o resultado do envio
            print('{}:{}: recebeu a mensagem'.format(i,nome))
            lista_enviados.append(date.today())
            
            # aguardar tempo randomico entre 10 e 15 segundos
            # isto é importante para evitar ter o numero bloqueado pelo Whatsapp pelo uso de automações
            time.sleep(random.randint(5,10))

        except:
            # imprimir no terminal o resultado do envio
            print('{}:{}: não recebeu a mensagem'.format(i,nome))
            lista_enviados.append('Falha no Envio')

print('Finalizado')

0:Izabel: recebeu a mensagem
1:Pablo: recebeu a mensagem
2:Acirema : recebeu a mensagem
4:Alan Catavento: recebeu a mensagem
5:Alberto Salão: recebeu a mensagem
7:Ana Carla Sampaio: recebeu a mensagem
9:Ana Luisa: recebeu a mensagem
10:Ana Mercia: recebeu a mensagem
11:Ana Oliva: recebeu a mensagem
12:Ana Paula : recebeu a mensagem
13:Ana Rosa: recebeu a mensagem
14:Anailza Meireles: recebeu a mensagem
15:Anamaria Romio: recebeu a mensagem
16:Andre: recebeu a mensagem
17:André Coêlho: recebeu a mensagem
18:André Ferreira : recebeu a mensagem
19:Andreia: recebeu a mensagem
20:Aninha: recebeu a mensagem
21:Aninha Junqueira: recebeu a mensagem
22:Ayna: recebeu a mensagem
23:Beatriz Bea: recebeu a mensagem
24:Beatriz Financeiro: recebeu a mensagem
25:Brena: recebeu a mensagem
26:Cacau : não recebeu a mensagem
28:Carla Galo : recebeu a mensagem
30:Carol Prima De Jana: recebeu a mensagem
31:Casa Mainha: recebeu a mensagem
32:Cecilia: recebeu a mensagem
34:Chica: recebeu a mensagem
35:Cidinho

In [8]:
lista_enviados

[datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 'Falha no Envio',
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10,

In [9]:
# copiar somente colunas Cliente e Telefone, apenas das que a mensagem esta não nula
resultado_df = contatos_df.loc[contatos_df['MENSAGEM'].notnull(),['CLIENTE','TELEFONE']]

In [10]:
resultado_df

,CLIENTE,TELEFONE
0,Izabel,55(71) 9308-2372
1,Pablo,55(71) 9101-3035
2,Acirema,55(71) 9170-4496
4,Alan Catavento,55(71) 9948-6196
5,Alberto Salão,55(71) 8774-4011
...,...,...
189,Erlon,55(71) 9943-6437
190,Simone Barbosa,55(71) 9605-0498
191,Zilca,55(71) 98817-9370
192,Carol,55(71) 98750-9231


In [11]:
# adicionar lista de resultado
resultado_df['MENSAGEM ENVIADA'] = lista_enviados

In [12]:
# mostrar o resultado
display(resultado_df)

,CLIENTE,TELEFONE,MENSAGEM ENVIADA
0,Izabel,55(71) 9308-2372,2022-10-01
1,Pablo,55(71) 9101-3035,2022-10-01
2,Acirema,55(71) 9170-4496,2022-10-01
4,Alan Catavento,55(71) 9948-6196,2022-10-01
5,Alberto Salão,55(71) 8774-4011,2022-10-01
...,...,...,...
189,Erlon,55(71) 9943-6437,2022-10-01
190,Simone Barbosa,55(71) 9605-0498,2022-10-01
191,Zilca,55(71) 98817-9370,2022-10-01
192,Carol,55(71) 98750-9231,2022-10-01


In [13]:
# salvar numa planilha
resultado_df.to_excel('Resultado Envios {}.xlsx'.format(date.today()),index=False)